## Introduction 

このレッスンでは以下を扱います: 
- 関数呼び出しとは何か、その使用例 
- OpenAIを使った関数呼び出しの作成方法 
- 関数呼び出しをアプリケーションに統合する方法 

## Learning Goals 

このレッスンを修了すると、以下のことができ理解できるようになります: 

- 関数呼び出しを使う目的 
- OpenAIサービスを使った関数呼び出しの設定 
- アプリケーションのユースケースに適した効果的な関数呼び出しの設計


## 関数呼び出しの理解

このレッスンでは、教育系スタートアップのために、ユーザーがチャットボットを使って技術コースを見つけられる機能を構築したいと考えています。ユーザーのスキルレベル、現在の役割、関心のある技術に合ったコースを推薦します。

これを実現するために、以下の組み合わせを使用します：
 - `OpenAI` を使ってユーザー向けのチャット体験を作成
 - `Microsoft Learn Catalog API` を使って、ユーザーのリクエストに基づいてコースを検索
 - `Function Calling` を使って、ユーザーのクエリを関数に送信し、APIリクエストを行う

まずは、なぜ関数呼び出しを使いたいのかを見てみましょう：

print("次のリクエストのメッセージ:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 関数の応答を参照できるGPTから新しい応答を取得


print(second_response.choices[0].message)


### なぜ関数呼び出しが必要か

このコースの他のレッスンを完了していれば、大規模言語モデル（LLM）を使うことの強力さはおそらく理解できているでしょう。おそらく、その限界のいくつかも見えているはずです。

関数呼び出しは、OpenAIサービスの機能であり、以下の課題に対応するために設計されています。

応答フォーマットの一貫性の欠如：
- 関数呼び出し以前は、大規模言語モデルからの応答は構造化されておらず、一貫性がありませんでした。開発者は出力の各バリエーションに対応するために複雑な検証コードを書く必要がありました。

外部データとの統合の制限：
- この機能が導入される前は、アプリケーションの他の部分からのデータをチャットコンテキストに組み込むことが困難でした。

応答フォーマットを標準化し、外部データとのシームレスな統合を可能にすることで、関数呼び出しは開発を簡素化し、追加の検証ロジックの必要性を減らします。

ユーザーは「ストックホルムの現在の天気は？」のような質問に答えを得ることができませんでした。これはモデルが訓練された時点のデータに制限されていたためです。

この問題を示す例を見てみましょう：

学生データのデータベースを作成し、適切なコースを提案できるようにしたいとします。以下には、含まれるデータが非常に似ている2人の学生の説明があります。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

このデータを解析するためにLLMに送信したいと考えています。これは後で、APIに送信したりデータベースに保存したりするためにアプリケーションで使用できます。

LLMに対して、私たちが関心のある情報について指示する、同一のプロンプトを2つ作成しましょう。


私たちはこれをLLMに送って、私たちの製品にとって重要な部分を解析してもらいたいと考えています。そこで、LLMに指示するために、同じプロンプトを2つ作成することができます。


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


これら二つのプロンプトを作成した後、`openai.ChatCompletion`を使ってそれらをLLMに送信します。プロンプトは`messages`変数に保存し、役割を`user`に割り当てます。これはユーザーからチャットボットへのメッセージが書かれていることを模倣するためです。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

これで、両方のリクエストをLLMに送信し、受け取った応答を確認できます。


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

プロンプトは同じで説明も似ていますが、`Grades` プロパティの形式が異なる場合があります。

上記のセルを複数回実行すると、形式が `3.7` または `3.7 GPA` になることがあります。

これは、LLMが書かれたプロンプトの形で非構造化データを受け取り、非構造化データを返すためです。データを保存または使用する際に何を期待すべきかを把握するために、構造化された形式が必要です。

関数呼び出しを使用することで、構造化されたデータを確実に受け取ることができます。関数呼び出しを使用する場合、LLMは実際に関数を呼び出したり実行したりするわけではありません。代わりに、LLMが応答で従うべき構造を作成します。その後、その構造化された応答を使用して、アプリケーションでどの関数を実行するかを判断します。


![関数呼び出しフローダイアグラム](../../../../translated_images/Function-Flow.083875364af4f4bb.ja.png)


次に、関数から返されたものを取得し、これをLLMに送り返すことができます。LLMはその後、自然言語を使ってユーザーの質問に答えます。


### 関数呼び出しの使用例

**外部ツールの呼び出し**  
チャットボットはユーザーからの質問に答えるのに優れています。関数呼び出しを使用することで、チャットボットはユーザーのメッセージを使って特定のタスクを完了できます。例えば、学生がチャットボットに「この科目についてもっと助けが必要だと講師にメールを送って」と頼むことができます。これは `send_email(to: string, body: string)` という関数呼び出しを行うことができます。

**APIやデータベースクエリの作成**  
ユーザーは自然言語を使って情報を見つけ、それがフォーマットされたクエリやAPIリクエストに変換されます。例えば、教師が「最後の課題を完了した学生は誰ですか」とリクエストすると、`get_completed(student_name: string, assignment: int, current_status: string)` という関数を呼び出すことができます。

**構造化データの作成**  
ユーザーはテキストのブロックやCSVを使い、LLMを使って重要な情報を抽出できます。例えば、学生が平和協定に関するWikipediaの記事を変換してAIフラッシュカードを作成することができます。これは `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` という関数を使って行うことができます。


## 2. 最初の関数呼び出しの作成

関数呼び出しを作成するプロセスは、主に3つのステップで構成されます：
1. 関数のリストとユーザーメッセージを使ってChat Completions APIを呼び出す
2. モデルの応答を読み取り、アクションを実行する（例：関数やAPIコールを実行する）
3. 関数からの応答を使ってユーザーへの応答を作成するために、Chat Completions APIを再度呼び出す


![関数呼び出しの流れ](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.ja.png)


### 関数呼び出しの要素

#### ユーザー入力

最初のステップはユーザーメッセージを作成することです。これはテキスト入力の値を動的に割り当てることもできますし、ここで値を割り当てることもできます。Chat Completions API を初めて使用する場合は、メッセージの `role` と `content` を定義する必要があります。

`role` は `system`（ルールの作成）、`assistant`（モデル）、または `user`（エンドユーザー）のいずれかです。関数呼び出しの場合、これを `user` として割り当て、例として質問を設定します。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 関数の作成。

次に、関数とその関数のパラメーターを定義します。ここでは `search_courses` という1つの関数だけを使用しますが、複数の関数を作成することもできます。

**重要** : 関数はシステムメッセージに含まれ、利用可能なトークン数に含まれます。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定義**

関数定義の構造は複数のレベルがあり、それぞれに固有のプロパティがあります。以下はネストされた構造の内訳です：

**トップレベルの関数プロパティ：**

`name` - 呼び出したい関数の名前。

`description` - 関数の動作の説明。ここでは具体的かつ明確であることが重要です。

`parameters` - モデルに応答で生成してほしい値と形式のリスト。

**パラメータオブジェクトのプロパティ：**

`type` - パラメータオブジェクトのデータ型（通常は "object"）

`properties` - モデルが応答で使用する特定の値のリスト。

**個別パラメータのプロパティ：**

`name` - プロパティキーによって暗黙的に定義される（例："role"、"product"、"level"）

`type` - この特定のパラメータのデータ型（例："string"、"number"、"boolean"）

`description` - 特定のパラメータの説明。

**オプションのプロパティ：**

`required` - 関数呼び出しを完了するために必要なパラメータを列挙した配列。


### 関数呼び出しの作成  
関数を定義した後、次にChat Completion APIへの呼び出しにそれを含める必要があります。これを行うには、リクエストに `functions` を追加します。この場合は `functions=functions` です。

また、`function_call` を `auto` に設定するオプションもあります。これは、ユーザーのメッセージに基づいてどの関数を呼び出すべきかをLLMに任せることを意味し、自分で割り当てるのではありません。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

では、レスポンスを見て、そのフォーマットがどのようになっているか見てみましょう：

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

関数の名前が呼び出されているのがわかり、ユーザーメッセージから、LLMが関数の引数に合うデータを見つけることができたことがわかります。


## 3.アプリケーションへの関数呼び出しの統合

LLMからのフォーマットされた応答をテストした後、これをアプリケーションに統合できます。

### フローの管理

これをアプリケーションに統合するために、次の手順を実行しましょう。

まず、OpenAIサービスへの呼び出しを行い、メッセージを`response_message`という変数に格納します。


In [ ]:
response_message = response.choices[0].message

これから、Microsoft Learn APIを呼び出してコースのリストを取得する関数を定義します:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



ベストプラクティスとして、まずモデルが関数を呼び出したいかどうかを確認します。その後、利用可能な関数の中から1つを作成し、呼び出されている関数に一致させます。  
次に、関数の引数を取得し、それらをLLMの引数にマッピングします。

最後に、関数呼び出しメッセージと`search_courses`メッセージによって返された値を追加します。これにより、LLMはユーザーに自然言語で応答するために必要なすべての情報を得ることができます。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



これで、更新されたメッセージをLLMに送信し、APIのJSON形式の応答ではなく自然言語の応答を受け取ることができます。


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## コードチャレンジ 

素晴らしい仕事です！OpenAI Function Callingの学習を続けるために、次のことに取り組んでみましょう：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - 学習者がより多くのコースを見つけるのに役立つ関数の追加パラメーター。利用可能なAPIパラメーターはこちらで確認できます： 
 - 学習者の母国語など、より多くの情報を取得する別の関数呼び出しを作成する 
 - 関数呼び出しやAPI呼び出しが適切なコースを返さない場合のエラーハンドリングを作成する 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の向上に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文の言語による文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
